In [1]:
##Import packages

import subprocess
import numpy as np
import os, sys, shutil
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd
import gc
import math
import time

from sklearn.cluster import KMeans

%matplotlib notebook

In [2]:
## Identify all water molecules
def distance(box, atom, aim_pool):
    vecs = abs(np.array(aim_pool, dtype='float')-np.array(atom, dtype='float'))
    for i in range(3):
        vecs[:,i] = np.minimum(vecs[:,i],box[i]-vecs[:,i])
    distance = np.linalg.norm(vecs,axis=1)
    return distance
def findNearest(box, atom, aim_pool, num):
    """
    Find the nearest one to given atom from a group of candidate
    Input: 
        box: periodic box size
        atom: given atom (position, dimention=3d)
        aim pool: a list of positions.
    Return:
        index of closest atom in aim_pool
    
    """
    vecs = abs(np.array(aim_pool, dtype='float')-np.array(atom, dtype='float'))
    for i in range(3):
        vecs[:,i] = np.minimum(vecs[:,i],box[i]-vecs[:,i])
    distance = np.linalg.norm(vecs,axis=1)
    sorted_dis = sorted(distance)
    res = np.where(distance <= sorted_dis[num])[0]
    if len(res)>num:
        res = np.concatenate((np.where(distance < sorted_dis[num])[0],np.where(distance == sorted_dis[num])[0]))
        res = res[:num]
    return res

In [3]:
### functions featuring h-bond.
def water_vec(water,box):
    o = np.array(water[0,1:],dtype='float')
    h1 = np.array(water[1,1:],dtype='float')
    h2 = np.array(water[2,1:],dtype='float')
    v1=h1-o
    v2=h2-o
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    v1 = __periodic(v1,box)
    v2 = __periodic(v2,box)
    return (v1+v2)/(np.linalg.norm(v1+v2))
def interactionE(w1, w2, box):
    ##TIP3P water model
    ##Unit real
    charge = {'O':-0.834, 'H':0.417}
    eps = {'OO': 0.1521, 'HH': 0.0460, 'HO': 0.0836}
    sig = {'OO': 3.1507, 'HH': 0.4000, 'HO': 1.7753}
    def __c__(eles, dis):
        e = 1.60217662e-19
        k = 8.987551792314e9 #N meter_e2 Ce-2
        Na = 6.02214076e23
        res = Na*k*charge[eles[0]]*charge[eles[1]]*e**2/(dis*1e-10)/1000
        ##unit: kJ/mol
        #print(res)
        return res
    def __lj__(eles, dis):
        e = eps[eles]
        s = sig[eles]
        return 4*e*((s/dis)**12-(s/dis)**6)*4.184
    c = 0
    lj=0
    for e1 in w1:
        for e2 in w2:
            dis = distance(box, e1[1:], [e2[1:]])[0]
            eles = ''.join(sorted(e1[0]+e2[0]))
            c+=__c__(eles, dis)
            lj+=__lj__(eles, dis)
    return c+lj
def OH_O_ang(w1, w2, box):
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    o1 = np.array(w1[0,1:],dtype='float')
    h1_1 = np.array(w1[1,1:],dtype='float')
    h1_2 = np.array(w1[2,1:],dtype='float')
    o2 = np.array(w2[0,1:],dtype='float')
    h2_1 = np.array(w2[1,1:],dtype='float')
    h2_2 = np.array(w2[2,1:],dtype='float')
    #print([h1_1, h1_2],findNearest(box, o2, [h1_1, h1_2], 1))
    H = [h1_1, h1_2][findNearest(box, o2, [h1_1, h1_2], 1)[0]]
    HO = __periodic(o1-H,box)
    HO = HO/np.linalg.norm(HO)
    H_O = __periodic(o2-H,box)
    H_O = H_O/np.linalg.norm(H_O)
    temp = np.dot(HO, H_O)
    if temp>1:
        temp=1
    if temp<-1:
        temp=-1
    return math.acos(temp)
def H_OH_ang(w1, w2, box):
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    o1 = np.array(w1[0,1:],dtype='float')
    h1_1 = np.array(w1[1,1:],dtype='float')
    h1_2 = np.array(w1[2,1:],dtype='float')
    o2 = np.array(w2[0,1:],dtype='float')
    h2_1 = np.array(w2[1,1:],dtype='float')
    h2_2 = np.array(w2[2,1:],dtype='float')
    H = [h1_1, h1_2][findNearest(box, o2, [h1_1, h1_2], 1)[0]]
    O_H = __periodic(H-o2,box)
    O_H = O_H/np.linalg.norm(O_H)
    OH1 = __periodic(h2_1-o2,box)
    OH1 = OH1/np.linalg.norm(OH1)
    OH2 = __periodic(h2_2-o2,box)
    OH2 = OH2/np.linalg.norm(OH2)
    t1 = np.dot(O_H, OH1)
    t2 = np.dot(O_H, OH2)
    if t1>1:
        t1=1
    if t1<-1:
        t1=-1
    if t2>1:
        t2=1
    if t2<-1:
        t2=-1
    return min([math.acos(t1), math.acos(t2)], key=abs)
def HOO_ang(w1, w2, box):
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    o1 = np.array(w1[0,1:],dtype='float')
    h1_1 = np.array(w1[1,1:],dtype='float')
    h1_2 = np.array(w1[2,1:],dtype='float')
    o2 = np.array(w2[0,1:],dtype='float')
    h2_1 = np.array(w2[1,1:],dtype='float')
    h2_2 = np.array(w2[2,1:],dtype='float')
    H = [h1_1, h1_2][findNearest(box, o2, [h1_1, h1_2], 1)[0]]
    O_H = __periodic(H-o1,box)
    O_H = O_H/np.linalg.norm(O_H)
    O_O = __periodic(o2-o1,box)
    O_O = O_O/np.linalg.norm(O_O)
    t1 = np.dot(O_H, O_O)
    if t1>1:
        t1=1
    if t1<-1:
        t1=-1
    return math.acos(t1)
def H_bondL(w1, w2, box):
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    o1 = np.array(w1[0,1:],dtype='float')
    h1_1 = np.array(w1[1,1:],dtype='float')
    h1_2 = np.array(w1[2,1:],dtype='float')
    o2 = np.array(w2[0,1:],dtype='float')
    h2_1 = np.array(w2[1,1:],dtype='float')
    h2_2 = np.array(w2[2,1:],dtype='float')
    H = [h1_1, h1_2][findNearest(box, o2, [h1_1, h1_2], 1)[0]]
    return np.linalg.norm(__periodic(H-o2,box))

In [4]:
## Identify Hydrogen Bond by energy and geometry

### defination of H bond
### OH_O bond ang >100 degree
### H_OH bond ang >70 degree
### interaction energy < -9.4kJ/mol
### OO distance <= 3.8
def id_Hbond(w1, w2, box):
    ## from w1(donar) to w2(acceptor) ( H bond by H from w1 and O from w2)
    """
    w1: array, [['O', Ox, Oy, Oz],['H', Hx, Hy, Hz],['H', Hx, Hy, Hz]]
    w1: array, [['O', Ox, Oy, Oz],['H', Hx, Hy, Hz],['H', Hx, Hy, Hz]]
    """
    #sol0: OO distance
    sol0 = distance(box, w1[0,1:], [w2[0,1:]])[0] <= 6
    #sol1: OH-O angle 
    sol1 = OH_O_ang(w1, w2, box) > math.pi/180.0*100
    #sol2: minimum of the two H-OH angle
    sol2 = H_OH_ang(w1, w2, box)>math.pi/180.0*70
    #sol3: interaction energy**
    sol3 = interactionE(w1, w2, box)<-3.6
    #sol4: backward OH-O angle
    sol4 = OH_O_ang(w2, w1, box) < math.pi/180.0*100
    #sol5: HO-O angle
    sol5 = HOO_ang(w1, w2, box) < math.pi/180.0*45
    #sol6: backward HO-O angle
    sol6 = HOO_ang(w2, w1, box) > math.pi/180.0*20
    #sol7: H---O distance
    return sol3&sol5&sol0#&sol5&sol6 & sol0 & sol1 & sol2 & sol3

In [5]:
def build_Hbond_mat(water_mols, box):
    mol_num = len(water_mols)
    res = np.zeros((mol_num, mol_num))
    for i in range(mol_num):
        for j in range(mol_num):
            if id_Hbond(water_mols[i], water_mols[j], box):
                res[i,j] = 1
                res[j,i] = -1
    return res
def Hbond_pairs(water_mols, box):
    res = []
    mol_num = len(water_mols)
    temp_mat = build_Hbond_mat(water_mols, box)
    for i in range(mol_num):
        for j in range(mol_num):
            if temp_mat[i,j] == 1:
                res.append([i,j])
    return np.array(res)
def show_water_net(water_mols, box, pairs=False):
    
    def mapping(orig_pts, box):
        new_pts = np.array(orig_pts)
        new_pts = np.concatenate((new_pts-[box[0],0,0,0], new_pts, new_pts+[box[0],0,0,0]),axis = 0)
        new_pts = np.concatenate((new_pts-[0,box[1],0,0], new_pts, new_pts+[0,box[1],0,0]),axis = 0)
        new_pts = np.concatenate((new_pts-[0,0,box[2],0], new_pts, new_pts+[0,0,box[2],0]),axis = 0)
        return new_pts
    def if_inArea(pos, area):
        res = True
        for p,a in zip(pos, area):
            res = res and ((p <= a[1]) & (p>=a[0]))
        return res
    
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    
    if type(pairs)==bool:
        pairs = build_Hbond_mat(water_mols, box)
    points = np.c_[np.array(water_mols[:,0,1:],dtype = 'float'),np.arange(0,len(water_mols))]
    points = mapping(points, box)
    l_range = np.c_[np.zeros(3)-4,box+4]
    inrange_idx=[]
    for idx, O_pos in enumerate(points):
        if if_inArea(O_pos[:3], l_range):
            inrange_idx.append(idx)
    
    points = np.array(sorted(points[inrange_idx],key = lambda x:x[-1]))

    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    xx, yy, zz = np.meshgrid([0,box[0]],
                [0,box[1]],
                [0,box[2]])
    ax.voxels(xx,yy,zz,np.ones(xx.shape)[:-1,:-1,:-1],facecolors=np.array([[[(0., 0., 0., 0.)]]]),edgecolors='k')

    
    ax.scatter(points[:,0],points[:,1],points[:,2],color='red',alpha=0.7)
    
    for p in pairs:
        _from = np.array([pt[:3] for pt in points if pt[-1] == p[0]])
        _vec = __periodic(water_mols[p[1],0,1:]-water_mols[p[0],0,1:],box)
        #print(np.linalg.norm(_vec))
        _vec = np.array([_vec]*len(_from))
        ax.quiver(_from[:,0],_from[:,1],_from[:,2],_vec[:,0],_vec[:,1],_vec[:,2],\
                  linewidths=1,arrow_length_ratio=0.3)
    
    max_range = np.array([points[:,0].max()-points[:,0].min(), points[:,1].max()-points[:,1].min(), points[:,2].max()-points[:,2].min()]).max() / 2.0

    mid_x = (points[:,0].max()+points[:,0].min()) * 0.5
    mid_y = (points[:,1].max()+points[:,1].min()) * 0.5
    mid_z = (points[:,2].max()+points[:,2].min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    ax.axis('off')
    plt.show()
    
    return

In [6]:
## show 2 water molecules (3D):
from mpl_toolkits.mplot3d import Axes3D
def print_mol(w1, w2,box):
    ### red: H bond donor
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec
    o1 = np.array(w1[0,1:],dtype='float')
    h1_1 = np.array(w1[1,1:],dtype='float')
    h1_2 = np.array(w1[2,1:],dtype='float')
    oh1_1 = __periodic(h1_1-o1,box)
    oh1_2 = __periodic(h1_2-o1,box)
    o2 = np.array(w2[0,1:],dtype='float')
    h2_1 = np.array(w2[1,1:],dtype='float')
    h2_2 = np.array(w2[2,1:],dtype='float')
    oh2_1 = __periodic(h2_1-o2,box)
    oh2_2 = __periodic(h2_2-o2,box)
    h2_1 = o2+oh2_1
    h2_2 = o2+oh2_2
    oo = __periodic(o2-o1,box)
    displace= oo - (o2-o1)
    o2 = o2+displace
    h2_1 = h2_1+displace
    h2_2 = h2_2+displace
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ps = np.array([o1, o2, h1_1, h1_2, h2_1, h2_2])
    ps[:,0] = ps[:,0] - ps[:,0].min()
    ps[:,1] = ps[:,1] - ps[:,1].min()
    ps[:,2] = ps[:,2] - ps[:,2].min()
    
    ax.scatter(ps[:1,0],ps[:1,1],ps[:1,2],color='red',alpha=1,s=1000)
    ax.scatter(ps[1:2,0],ps[1:2,1],ps[1:2,2],color='pink',alpha=1,s=1000)
    ax.scatter(ps[2:,0],ps[2:,1],ps[2:,2],color='blue',alpha=1,s=800)
    
    X, Y, Z = [ps[:1,0]],[ps[:1,1]],[ps[:1,2]]
    U, V, W = [oo[0]],[oo[1]],[oo[2]]
    ax.quiver(X,Y,Z,U,V,W,)
    
    max_range = np.array([ps[:,0].max()-ps[:,0].min(), ps[:,1].max()-ps[:,1].min(), ps[:,2].max()-ps[:,2].min()]).max() / 2.0

    mid_x = (ps[:,0].max()+ps[:,0].min()) * 0.5
    mid_y = (ps[:,1].max()+ps[:,1].min()) * 0.5
    mid_z = (ps[:,2].max()+ps[:,2].min()) * 0.5
    ax.set_xlim(mid_x - max_range-2, mid_x + max_range+2)
    ax.set_ylim(mid_y - max_range-2, mid_y + max_range+2)
    ax.set_zlim(mid_z - max_range-2, mid_z + max_range+2)
    print('red: H bond donor')
    ax.axis('off')
    plt.show()

In [7]:
'''
def findPath(pairs, start):
    pairs = np.array(pairs)
    paths = [[start]]
    ends=[[start]]
    _j = True
    while _j:
        for i, _s in enumerate(ends):
            if _s == False:
                continue
            if _s not in path[:,0]:
                ends[_i] = 
'''
def adjacency_Mat(water_mols, box, no_dir = False):
    mol_num = len(water_mols)
    res = np.zeros((mol_num, mol_num))
    for i in range(mol_num):
        for j in range(mol_num):
            #print(i,j)
            if id_Hbond(water_mols[i], water_mols[j], box):
                res[i,j] = 1
                if no_dir:
                    res[j,i] = 1
    return res

In [8]:
### Distance Mat
def ini_d_mat(mol_num, adj_mat):
    res = np.array([[float('inf')]*mol_num]*mol_num)
    for i in range(mol_num):
        for j in range(mol_num):
            
            if i==j:
                res[i,j] = 0
            if adj_mat[i,j]==1:
                res[i,j]=1
    return res
def Floyd(d_mat):
    for k in range(len(d_mat)):
        for i in range(len(d_mat)):
            for j in range(len(d_mat)):
                d_mat[i,j] = min([d_mat[i,j], d_mat[i,k]+d_mat[k,j]])
    return d_mat

In [9]:
def get_OH(atoms_lines, box):
    
    pos = [[float(_e) for _e in e.split()] for e in atoms_lines[1:]]
    pos = np.array(pos, dtype = 'object')
    atoms = np.concatenate((eles, pos),axis=1)
    for i in range(len(box)):
        atoms[:,i+1] = atoms[:,i+1]*box[i]
    
    Os = atoms[np.where(atoms[:,0]=='O')[0],1:]
    Hs = atoms[np.where(atoms[:,0]=='H')[0],1:]

    dis_mat = np.zeros((len(Hs), len(Os)))
    for i in range(len(Hs)):
        dis_mat[i] = distance(box, Hs[i], Os)
    dis = np.array([min(e) for e in dis_mat])
    return dis

In [10]:
def water_form(lines, eles, box, res_type = 'idxs'):
    mol_type_class = {'-OH in H3PO4':6, '-O in H3PO4':7, 'hydronium-AAA':4, 
                        'water-DDA':2, 'water-DA':0, 'water-DDAA':3, 'water-DAA':1}
    
    pos = [[float(_e) for _e in e.split()] for e in lines[1:]]
    pos = np.array(pos, dtype = 'object')
    atoms = np.concatenate((eles, pos),axis=1)
    for i in range(len(box)):
        atoms[:,i+1] = atoms[:,i+1]*box[i]
        
    Oinds = np.where(atoms[:,0]=='O')[0]
    Hinds = np.where(atoms[:,0]=='H')[0]
    Pinds = np.where(atoms[:,0]=='P')[0]

    Os = atoms[Oinds,1:]
    Hs = atoms[Hinds,1:]
    Ps = atoms[Pinds,1:]

    OH_cutoff = 1.28
    O_H_cutoff = 2.25   
        
    mol = []
    for i in range(len(Os)):
        temp_OH = distance(box, Os[i], Hs)
        OH_idx = np.where(temp_OH<OH_cutoff)[0]
        No_OH = len(OH_idx)
        No_O_H = len(np.where(temp_OH<O_H_cutoff)[0]) - No_OH
        t = []
        t.append(No_OH)
        t.append(No_O_H)
        doner_H = 0
        #print(i)
        for _H in OH_idx:
            temp_HO = distance(box, Hs[_H], Os)
            #print(sorted(temp_HO))
            temp_HO = np.where(temp_HO<O_H_cutoff)[0]
            doner_H += (len(temp_HO)-1)
        #print(doner_H)
        t.append(doner_H)
        mol.append(t)
    mol = np.array(mol,dtype='object')
    
    mol_type = ['']*len(Os)
    for i, wat in enumerate(mol):
        temp = ''
        if wat[0] == 3:
            temp = temp+'hydronium'
            if wat[1] == 0 and wat[2] == 3:
                temp = temp+'-AAA'
            else:
                temp = temp+'-exception'
        elif wat[0] == 2:
            temp = temp+'water'
            if wat[2] > 0:
                temp = temp+'-'+'D'*wat[2]
            if wat[1] > 0:
                temp = temp+'A'*wat[1]
        else:
            temp = '-O'
            if wat[0]>0:
                temp = temp+'H'
            temp = temp+' in H3PO4'
        if res_type == 'idxs':
            if temp not in mol_type_class:
                mol_type[i] = 5
            else:
                mol_type[i] = mol_type_class[temp]
        else:
            mol_type[i] = temp
        
    return mol_type

In [11]:
def get_O_in_OH(atoms_lines, box):
    
    pos = [[float(_e) for _e in e.split()] for e in atoms_lines[1:]]
    pos = np.array(pos, dtype = 'object')
    atoms = np.concatenate((eles, pos),axis=1)
    for i in range(len(box)):
        atoms[:,i+1] = atoms[:,i+1]*box[i]
    
    Os = atoms[np.where(atoms[:,0]=='O')[0],1:]
    Hs = atoms[np.where(atoms[:,0]=='H')[0],1:]

    dis_mat = np.zeros((len(Hs), len(Os)))
    for i in range(len(Hs)):
        dis_mat[i] = distance(box, Hs[i], Os)
    dis = np.array([np.where(e == min(e))[0][0] for e in dis_mat])
    return dis

In [12]:
## Example structure
#### read structure
#data_path = '/bigdata/greaneylab/wzhan097/whoochul'
data_path = '/bigdata/greaneylab/wzhan097/woochul'
filename = 'H3PO4_lowT_20ps.vasp'

data_path = '/rhome/wzhan097/shared/H3PO4_XDATCAR/LowT_last_run'
filename = 'XDATCAR'
with open(os.path.join(data_path,filename),'r') as f:
    lines = f.readlines()
box = lines[2:5]
box =[e.split() for e in box]
box = [box[0][0],box[1][1],box[2][2]]
box = [float(e) for e in box]
lines = lines[5:]
types = lines[0].split()
temp = lines[1].split()
temp = [int(e) for e in temp]
eles = []
atom_num = sum(temp)

for _i,_j in zip(types, temp):
    eles = eles+[[_i]]*_j
eles = np.array(eles, dtype = 'object')
lines = lines[2:]

In [13]:
####The formation of water is changing

O_H_mat = np.zeros((atom_num, atom_num))

pos = [[float(_e) for _e in e.split()] for e in lines[1:1+atom_num]]
pos = np.array(pos, dtype = 'object')
atoms = np.concatenate((eles, pos),axis=1)

for i in range(len(box)):
    atoms[:,i+1] = atoms[:,i+1]*box[i]

Oinds = np.where(atoms[:,0]=='O')[0]
Hinds = np.where(atoms[:,0]=='H')[0]

for i in range( 0,int(len(lines)/(atom_num+1)) ):
    temp = get_O_in_OH(lines[i*(atom_num+1):(i+1)*(atom_num+1)],box)
    for _i, _j in enumerate(temp):
        O_H_mat[Oinds[_j], Hinds[_i]]+=1
    if i%100 == 0:
        print(i)

0
100
200


KeyboardInterrupt: 

In [ ]:
bond_Os, bond_Hs = np.where(O_H_mat!=0)
bond_l = len(bond_Os)
bond_a = 0
for _O in set(bond_Os):
    bond_a += sum(list(range(np.count_nonzero(bond_Os == _O, axis=0))))
bond_l = np.zeros(((int(len(lines)/(atom_num+1))), bond_l))
bond_a = np.zeros(((int(len(lines)/(atom_num+1))), bond_a))

In [ ]:
bond_Os, bond_Hs

In [ ]:
def _if_bond_(lines, Os, Hs, box):
    
    OH_cutoff = 1.28
    
    pos = [[float(_e) for _e in e.split()] for e in lines[1:]]
    pos = np.array(pos, dtype = 'object')
    for i in range(len(box)):
        pos[:,i] = pos[:,i]*box[i]
    
    i = 0
    l = []
    ang = []
    
    
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec

    def __angle(vec1, vec2):
        a = vec1/np.linalg.norm(vec1)
        b = vec2/np.linalg.norm(vec2)
        temp = np.dot(vec1, vec2)
        if temp>1:
            temp=1
        if temp<-1:
            temp=-1
        return math.acos(temp)
    
    while i < len(Os):
        _O = Os[i]
        _num = np.count_nonzero(Os == _O, axis=0)
        vec = []
        for j in range(_num):
            _i = i+j 
            _temp = np.linalg.norm(__periodic(pos[Hs[_i]]-pos[Os[_i]],box))
            if _temp <= OH_cutoff:
                l.append(1)
                vec.append(1)
            else:
                l.append(0)
                vec.append(0)
        if _num > 1:
            temp = np.zeros((_num,_num))
            for _i in range(_num):
                for _j in range(_i+1,_num):
                    ang.append(vec[_i]*vec[_j])
        i=i+_num
    return np.array(l), np.array(ang)
                    
        

In [ ]:
def bond_angle(lines, Os, Hs, box):
    
    pos = [[float(_e) for _e in e.split()] for e in lines[1:]]
    pos = np.array(pos, dtype = 'object')
    for i in range(len(box)):
        pos[:,i] = pos[:,i]*box[i]
    
    i = 0
    l = []
    ang = []
    
    
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec

    def __angle(vec1, vec2):
        a = vec1/np.linalg.norm(vec1)
        b = vec2/np.linalg.norm(vec2)
        temp = np.dot(vec1, vec2)
        if temp>1:
            temp=1
        if temp<-1:
            temp=-1
        return math.acos(temp)
    
    while i < len(Os):
        _O = Os[i]
        _num = np.count_nonzero(Os == _O, axis=0)
        vec = []
        for j in range(_num):
            _i = i+j
            vec.append(__periodic(pos[Hs[_i]]-pos[Os[_i]],box)) 
            l.append(np.linalg.norm(vec[-1]))
        if _num > 1:
            temp = np.zeros((_num,_num))
            for _i in range(_num):
                for _j in range(_i+1,_num):
                    ang.append(__angle(vec[_i],vec[_j]))
        i=i+_num
    return np.array(l), np.array(ang)
                    
        

In [ ]:
_x,_y = _if_bond_(lines[1*(atom_num+1):(1+1)*(atom_num+1)], bond_Os, bond_Hs, box)
_x,_y

In [ ]:
for i in range( 0,int(len(lines)/(atom_num+1)) ):
    _x,_y = _if_bond_(lines[i*(atom_num+1):(i+1)*(atom_num+1)], bond_Os, bond_Hs, box)
    bond_l[i] = _x
    bond_a[i] = _y
    if i%100 == 0:
        print(i)

In [ ]:
bond_l, bond_a

In [ ]:
outfile = "RT_last_run_OH_flg.npy"
np.save(outfile,bond_l)
outfile = "RT_last_run_OHO_flg.npy"
np.save(outfile,bond_a)

In [ ]:
names = ["RT_last_run_OH_flg","RT_last_run_OHO_flg"]
for name in names:
    x = np.load(name+'.npy')
    lines = ['\t'.join(list(map(str,e)))+'\n' for e in x]
    with open(name+'.tsv','w+') as f:
        f.writelines(lines)

In [ ]:
pos = [[float(_e) for _e in e.split()] for e in lines[1:1+atom_num]]
pos = np.array(pos, dtype = 'object')
atoms = np.concatenate((eles, pos),axis=1)
for i in range(len(box)):
    atoms[:,i+1] = atoms[:,i+1]*box[i]

In [ ]:
Oinds = np.where(atoms[:,0]=='O')[0]
Hinds = np.where(atoms[:,0]=='H')[0]
Pinds = np.where(atoms[:,0]=='P')[0]

Os = atoms[Oinds,1:]
Hs = atoms[Hinds,1:]
Ps = atoms[Pinds,1:]

In [ ]:
OH_cutoff = 1.28
O_H_cutoff = 2.25

In [ ]:
dis_mat = np.zeros((int(len(lines)/(atom_num+1)),len(np.where(eles=='H')[0])))
dis_mat.shape

In [ ]:
for i in range(int(len(lines)/(atom_num+1))):
    temp = get_OH(lines[i*(atom_num+1):(i+1)*(atom_num+1)],box)
    dis_mat[i] = temp

In [ ]:
get_mol(lines[0*(atom_num+1):(0+1)*(atom_num+1)], box)

In [ ]:
mol_type_time = np.zeros((int(len(lines)/(atom_num+1)), len(np.where(eles=='O')[0])))
for i in range( 0,int(len(lines)/(atom_num+1)) ):
    temp = water_form(lines[i*(atom_num+1):(i+1)*(atom_num+1)], eles, box)
    mol_type_time[i] = temp
    if i%100 == 0:
        print(i)

In [ ]:
mol_type_time

0: water-DA
1: water-DAA
2: water-DDA
3: water-DDAA
4: hydronium-AAA
5: Other
6: -OH in H3PO4
7: -O in H3PO4

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

####RT
#RT_mol_type_time = mol_type_time
ax = sns.heatmap(RT_mol_type_time.T,cmap='RdBu_r')
plt.show()

In [ ]:
ax = sns.heatmap(RT_mol_type_time.T[:,-200:],cmap='RdBu_r')
plt.show()

In [ ]:
######LowT
#LowT_mol_type_time = mol_type_time
ax = sns.heatmap(LowT_mol_type_time.T,cmap='RdBu_r')
plt.show()

In [ ]:
ax = sns.heatmap(LowT_mol_type_time.T[:,-200:],cmap='RdBu_r')
plt.show()

In [ ]:
plt.scatter(x = range(len(RT_mol_type_time[:,78])),y=RT_mol_type_time[:,78])

In [ ]:
#H3PO4_highT_20ps
#mol_type_highT = mol_type

mol_type_highT_class = {'-OH in H3PO4':0, '-O in H3PO4':0, 'hydronium-AAA':0,
                        'water-DDA':0, 'water-DA':0, 'water-DDAA':0, 'water-DAA':0}
for e in mol_type_highT_class:
    mol_type_highT_class[e] = mol_type_highT.count(e)
mol_type_highT_class

In [ ]:
#H3PO4_lowT_20ps
#mol_type_lowT = mol_type

mol_type_lowT_class = {'-OH in H3PO4':0, '-O in H3PO4':0, 'hydronium-AAA':0,
                        'water-DDA':0, 'water-DA':0, 'water-DDAA':0, 'water-DAA':0}
for e in mol_type_lowT_class:
    mol_type_lowT_class[e] = mol_type_lowT.count(e)
mol_type_lowT_class